In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from sklearn.model_selection import KFold
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import openpyxl
from tqdm import tqdm  
from googletrans import Translator
# Translation
from tqdm import tqdm  
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
# from collections import Counter
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
# from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import KFold
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
from bidi.algorithm import get_display

[nltk_data] Downloading package stopwords to C:\Users\Hasibul
[nltk_data]     Hamim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Hasibul
[nltk_data]     Hamim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load Data
wb = openpyxl.load_workbook('E:/Research Work/Bangla_CyberBulling_2024/Code & dataset/Dataset/Final_Dataset_94k_Bangla_CyberBulling_Combine_Dataset_94k.xlsx')
ws = wb['Sheet1']
data_rows = []
for row in ws['A1':'B94000']:
    data_cols = [cell.value for cell in row]
    data_rows.append(data_cols)
df = pd.DataFrame(data_rows)
header = df.iloc[0]
df.columns = [header]
df = df.iloc[1:, :]

C:\Users\Hasibul Hamim\AppData\Local\Temp\ipykernel_13800\2492820922.py:10: DeprecationWarning: Series._data is deprecated and will be removed in a future version. Use public APIs instead.
  df.columns = [header]


In [3]:
class_counts = df['label'].value_counts()
print("Class Counts:\n", class_counts)

Class Counts:
 (label,) 
not bully    33578
troll        23193
sexual       18026
religious    15424
threat        3778
Name: count, dtype: int64


In [4]:
# all_comments = ' '.join(df['comment'].apply(lambda x: ' '.join(map(str, x))).astype(str))

# rgx = r"[\u0980-\u09FF]+"
# wordcloud = WordCloud(font_path='E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/Dataset/Font/CHANO___.ttf',regexp=rgx, width=2000, height=1300, background_color='white').generate(all_comments)

# plt.figure(figsize=(30, 15))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.show()

In [5]:
# Data Exploration
print(df.head())
print(df.iloc[:, -1].value_counts())
print(df.shape)
print(df.isna().sum())

# Handle Missing Values
mode = df.iloc[:, -2].value_counts().index[0]
df.iloc[:, -2].fillna(mode, inplace=True)
print(df.isna().sum())
print(df.head())

0                                            comment      label
1  হয়তো ❤ আয়মান ভাইয়ের পেইজের এডমিন $% মুনজেরিন আ...      troll
2              এতো :) আমাদের জন্য বড় # আনন্দের সংবাদ      troll
3  >এজন্যই বলি আ-লীগে ❤ চলে আসুন। সব নির্বাচন-ই স...  not bully
4  এটিএম :) কার্ড দেখিয়ে স্টুডেন্ট # হাফপাস নেওয়া...      troll
5                       > আমাদের জীবনে কোম সিজি নাই?      troll
(label,)
not bully    33578
troll        23193
sexual       18026
religious    15424
threat        3778
Name: count, dtype: int64
(93999, 2)
comment    0
label      0
dtype: int64
comment    0
label      0
dtype: int64
0                                            comment      label
1  হয়তো ❤ আয়মান ভাইয়ের পেইজের এডমিন $% মুনজেরিন আ...      troll
2              এতো :) আমাদের জন্য বড় # আনন্দের সংবাদ      troll
3  >এজন্যই বলি আ-লীগে ❤ চলে আসুন। সব নির্বাচন-ই স...  not bully
4  এটিএম :) কার্ড দেখিয়ে স্টুডেন্ট # হাফপাস নেওয়া...      troll
5                       > আমাদের জীবনে কোম সিজি নাই?      troll


In [6]:
# Labeling and Encoding
list(df.iloc[:, -1].value_counts().index)
df['nlabel'] = df.label.replace(['not bully', 'troll', 'sexual', 'religious', 'threat'], [0, 1, 2, 3, 4])
df['noutput'] = df.iloc[:, -1].replace(['bully', 'normal'], [1, 0])
print(df.head())
print(df.head())

0                                            comment      label nlabel noutput
1  হয়তো ❤ আয়মান ভাইয়ের পেইজের এডমিন $% মুনজেরিন আ...      troll      1       1
2              এতো :) আমাদের জন্য বড় # আনন্দের সংবাদ      troll      1       1
3  >এজন্যই বলি আ-লীগে ❤ চলে আসুন। সব নির্বাচন-ই স...  not bully      0       0
4  এটিএম :) কার্ড দেখিয়ে স্টুডেন্ট # হাফপাস নেওয়া...      troll      1       1
5                       > আমাদের জীবনে কোম সিজি নাই?      troll      1       1
0                                            comment      label nlabel noutput
1  হয়তো ❤ আয়মান ভাইয়ের পেইজের এডমিন $% মুনজেরিন আ...      troll      1       1
2              এতো :) আমাদের জন্য বড় # আনন্দের সংবাদ      troll      1       1
3  >এজন্যই বলি আ-লীগে ❤ চলে আসুন। সব নির্বাচন-ই স...  not bully      0       0
4  এটিএম :) কার্ড দেখিয়ে স্টুডেন্ট # হাফপাস নেওয়া...      troll      1       1
5                       > আমাদের জীবনে কোম সিজি নাই?      troll      1       1


In [7]:
translator = Translator()
output = []

for i in tqdm(range(len(df))):
    if df.label.values[i] == 'not bully':
        output.append('Normal')
    else:
        output.append('Bully')

df['output'] = output
df['noutput'] = df.iloc[:, -1].replace(['Bully', 'Normal'], [1, 0])


  0%|          | 0/93999 [00:00<?, ?it/s]

100%|██████████| 93999/93999 [01:16<00:00, 1228.23it/s]


In [8]:
df['comment']

,comment
1,হয়তো ❤ আয়মান ভাইয়ের পেইজের এডমিন $% মুনজেরিন আ...
2,এতো :) আমাদের জন্য বড় # আনন্দের সংবাদ
3,>এজন্যই বলি আ-লীগে ❤ চলে আসুন। সব নির্বাচন-ই স...
4,এটিএম :) কার্ড দেখিয়ে স্টুডেন্ট # হাফপাস নেওয়া...
5,> আমাদের জীবনে কোম সিজি নাই?
...,...
93995,হিরো আলম হিরো আলম আছে থাকবে ইনশাল্লাহ জনদরদি গ...
93996,"হিরো আলম,, এগিয়ে যাও"
93997,হিরো আলমকে সাপোর্ট দেওয়ার জন্য অসংখ্য ধন্যবাদ...
93998,হিরো ভাই তুমি এগিয়ে য়াও


In [9]:
df.to_csv('v1.csv')

In [10]:
print(stopwords.fileids())
sw = stopwords.words('bengali')

new_stopwords = stopwords.words('english')
new_stopwords.append('SampleWord')

df.to_csv('df_english1000.csv')

nltk.download('wordnet')
nltk.download('omw-1.4')

wordnet_lemmatizer = WordNetLemmatizer()

M = len(df)
for k in tqdm(range(M)):
    text = df.iloc[:, 0].values[k]
    txt = re.sub(r'http\S+', '', text)
    txt = re.sub(r'[!@#$%^&*?><,./\-+`~|:);(❤}{]', '', txt)

    text_tokens = word_tokenize(txt)

    def detect_benglish(text_tokens):
        benglish = False
        for j in range(len(text_tokens)):
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        benglish = True
        return benglish

    def handle_benglish(text_tokens):
        for j in range(len(text_tokens)):
            s = []
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        s.append(i)
                    if txt[i] == '|':
                        s.append(i)
            for i in range(len(s)):
                txt = txt.replace(txt[s[i]], txt[s[i]] + " ")
            text_tokens[j] = txt
        tstring = str(' '.join(text_tokens)).lower()
        text_tokens = word_tokenize(tstring)
        return text_tokens

    remove_sw = [word for word in text_tokens if not word in sw]
    un_items = np.unique(remove_sw)
    r_sw = [wordnet_lemmatizer.lemmatize(w) for w in un_items]
    bn_tokens = []

    def tanslate_bengali(r_sw):
        for i in range(len(r_sw)):
            bn_tokens.append(translator.translate(r_sw[i], dest='bn').text)
        return bn_tokens

    bn_token = r_sw
    df.iloc[:, 0].values[k] = ' '.join(bn_token)

df.iloc[:, -1].value_counts()
df.iloc[:, 0]

df[:].iloc[:, 0].shape


['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


[nltk_data] Downloading package wordnet to C:\Users\Hasibul
[nltk_data]     Hamim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Hasibul
[nltk_data]     Hamim\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 93999/93999 [00:51<00:00, 1814.94it/s]


(93999,)

In [11]:
df

,comment,label,nlabel,noutput,output
1,আইডি আপু। আয়মান এডমিন গিয়েছেন। পেইজের ভাইয়ের ভ...,troll,1,1,Bully
2,আনন্দের এতো বড় সংবাদ,troll,1,1,Bully
3,আলীগে আসুন। এজন্যই নির্বাচন। নির্বাচনই বলি সুষ্ঠু,not bully,0,0,Normal
4,এটিএম কার্ড দেখিয়ে স্টুডেন্ট হাফপাস,troll,1,1,Bully
5,কোম জীবনে সিজি,troll,1,1,Bully
...,...,...,...,...,...
93995,অবশ্যই আছি আমরাও আলম ইনশাআল্লাহ ইনশাল্লাহ করবো...,not bully,0,0,Normal
93996,আলম এগিয়ে যাও হিরো,not bully,0,0,Normal
93997,অসংখ্য আপনাকে আলমকে ধন্যবাদ সাপোর্ট হিরো,not bully,0,0,Normal
93998,এগিয়ে ভাই হিরো য়াও,not bully,0,0,Normal


In [37]:
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.linear_model import LogisticRegression
# from scipy.stats import f_oneway
# from imblearn.under_sampling import InstanceHardnessThreshold
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# import numpy as np

# # Text Vectorization (Count)
# count_vectorizer = CountVectorizer(max_features=20000, tokenizer=word_tokenize, stop_words=stopwords.words('english'))
# iX_count = count_vectorizer.fit_transform(df.iloc[:5000, 0].values)

# # TF-IDF Transformation
# tfidf_transformer = TfidfTransformer()
# iX_tfidf = tfidf_transformer.fit_transform(iX_count)

# # Perform ANOVA test for feature selection
# anova_selector = SelectKBest(score_func=f_classif, k=1000)
# iX_tfidf_selected = anova_selector.fit_transform(iX_tfidf, df.iloc[:5000, -3].values)

# # Convert sparse matrix to dense array
# iX_tfidf_selected_dense = iX_tfidf_selected.toarray()

# # Perform ANOVA test on the entire dataset
# anova_result = f_oneway(*[iX_tfidf_selected_dense[:, i] for i in range(iX_tfidf_selected_dense.shape[1])])
# print("\nANOVA Results:")
# print("F-value:", anova_result.statistic)
# print("p-value:", anova_result.pvalue)
# print("Selected Features Shape:", iX_tfidf_selected_dense.shape)

# # Cross-Validation
# kf = KFold(n_splits=5, random_state=42, shuffle=True)
# for train_index, test_index in kf.split(iX_tfidf_selected_dense):
#     X_train, X_test = iX_tfidf_selected_dense[train_index], iX_tfidf_selected_dense[test_index]
#     y_train, y_test = df.iloc[:5000, -3].values[train_index], df.iloc[:5000, -3].values[test_index]

#     # Logistic Regression (or any other classifier) and evaluation code here


In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from scipy.stats import f_oneway
from imblearn.under_sampling import InstanceHardnessThreshold
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np

# Text Vectorization (Count)
count_vectorizer = CountVectorizer(max_features=20000, tokenizer=word_tokenize, stop_words=stopwords.words('english'))
iX_count = count_vectorizer.fit_transform(df.iloc[:5000, 0].values)

# TF-IDF Transformation
tfidf_transformer = TfidfTransformer()
iX_tfidf = tfidf_transformer.fit_transform(iX_count)

# Perform ANOVA test for feature selection
anova_selector = SelectKBest(score_func=f_classif, k=1000)
iX_tfidf_selected = anova_selector.fit_transform(iX_tfidf, df.iloc[:5000, -3].values)

# Imbalanced Data Handling
cc = InstanceHardnessThreshold(random_state=10, estimator=LogisticRegression())
X_resampled, y_resampled = cc.fit_resample(iX_tfidf_selected, df.iloc[:5000, -3].values)

# Convert sparse matrix to dense array
X_resampled_dense = X_resampled.toarray()

# Perform ANOVA test on the entire dataset
anova_result = f_oneway(*[X_resampled_dense[:, i] for i in range(X_resampled_dense.shape[1])])
print("\nANOVA Results:")
print("F-value:", anova_result.statistic)
print("p-value:", anova_result.pvalue)
print("Selected Features Shape:", X_resampled_dense.shape)

# Cross-Validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X_resampled_dense):
    X_train, X_test = X_resampled_dense[train_index], X_resampled_dense[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]

    # Logistic Regression (or any other classifier) and evaluation code here


c:\Users\Hasibul Hamim\anaconda3\envs\tf\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Hasibul Hamim\anaconda3\envs\tf\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(



ANOVA Results:
F-value: 20.988634075338258
p-value: 0.0
Selected Features Shape: (2250, 1000)


In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from scipy.stats import chi2_contingency
from imblearn.under_sampling import InstanceHardnessThreshold
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np

# Text Vectorization (Count)
count_vectorizer = CountVectorizer(max_features=20000, tokenizer=word_tokenize, stop_words=stopwords.words('english'))
iX_count = count_vectorizer.fit_transform(df.iloc[:5000, 0].values)

# TF-IDF Transformation
tfidf_transformer = TfidfTransformer()
iX_tfidf = tfidf_transformer.fit_transform(iX_count)

# Perform chi2 test for feature selection
chi2_selector = SelectKBest(score_func=chi2, k=1000)
iX_tfidf_selected = chi2_selector.fit_transform(iX_tfidf, df.iloc[:5000, -3].values)

# Imbalanced Data Handling
cc = InstanceHardnessThreshold(random_state=10, estimator=LogisticRegression())
X_resampled, y_resampled = cc.fit_resample(iX_tfidf_selected, df.iloc[:5000, -3].values)

# Convert sparse matrix to dense array
X_resampled_dense = X_resampled.toarray()

# Add a small value to the contingency table to avoid zero element
X_resampled_dense += np.random.rand(*X_resampled_dense.shape) * 0.0001

# Perform chi2 test on the entire dataset
chi2_result = chi2_contingency(X_resampled_dense)
print("\nchi2 Results:")
print("chi2-value:", chi2_result[0])
print("p-value:", chi2_result[1])
print("Selected Features Shape:", X_resampled_dense.shape)

# Cross-Validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X_resampled_dense):
    X_train, X_test = X_resampled_dense[train_index], X_resampled_dense[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]

    # Logistic Regression (or any other classifier) and evaluation code here


c:\Users\Hasibul Hamim\anaconda3\envs\tf\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Hasibul Hamim\anaconda3\envs\tf\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(



chi2 Results:
chi2-value: 825999.7600932752
p-value: 1.0
Selected Features Shape: (2250, 1000)
